# Enriching a BEL Graph with the CTD

**Author:** [Charles Tapley Hoyt](https://github.com/cthoyt)

This notebook outlines a short process for enriching the Sialic Acid example graph with gene, mRNA, and protein interactions from the Chemotoxicogenomics Database (CTD).

## Imports

In [1]:
import itertools as itt
import sys
import time

from bio2bel_ctd import Manager as CTDManager
from bio2bel_entrez import Manager as EntrezManager
from bio2bel_hgnc import Manager as HGNCManager
from bio2bel_hgnc.enrich import add_orthologies

from pybel.constants import *
from pybel.examples import sialic_acid_graph
from pybel_tools.mutation import infer_central_dogma
from pybel_tools.visualization import to_jupyter

In [2]:
import logging

logging.basicConfig(level=logging.DEBUG, format='%(message)s')
logging.getLogger('bio2bel_ctd.enrichment_utils').setLevel(logging.DEBUG)

## Runtime

In [3]:
print(sys.version)

3.6.5 (default, Apr 25 2018, 14:23:58) 
[GCC 4.2.1 Compatible Apple LLVM 9.1.0 (clang-902.0.39.1)]


In [4]:
print(time.asctime())

Mon Apr 30 11:42:28 2018


## Managers

Several Bio2BEL repositories are used to accomplish this, including HGNC, Entrez, and CTD.

In [5]:
ctd_manager = CTDManager()

fetching global bio2bel config from /Users/cthoyt/.pybel/bio2bel/config.ini
load default connection string from mysql+mysqldb://root@localhost/pybel?charset=utf8


In [6]:
entrez_manager = EntrezManager()

fetching global bio2bel config from /Users/cthoyt/.pybel/bio2bel/config.ini
load default connection string from mysql+mysqldb://root@localhost/pybel?charset=utf8


In [7]:
hgnc_manager = HGNCManager()

fetching global bio2bel config from /Users/cthoyt/.pybel/bio2bel/config.ini
load default connection string from mysql+mysqldb://root@localhost/pybel?charset=utf8


# Workflow

## Starting Graph

In [8]:
to_jupyter(sialic_acid_graph)

<IPython.core.display.Javascript object>

## Enrich Equivalencies and Ontological Relationships

Add all central dogma mRNA and genes to the proteins mentioned

In [9]:
infer_central_dogma(sialic_acid_graph)

Add all of the Entrez equivalent nodes for the gene, mRNA, and protein nodes present

In [10]:
hgnc_manager.enrich_hgnc_with_entrez_equivalences(sialic_acid_graph)

In [11]:
to_jupyter(sialic_acid_graph)

<IPython.core.display.Javascript object>

## Enrich Genes with CTD

In [12]:
%%time
ctd_manager.enrich_graph_genes(sialic_acid_graph)

did not map (246485) arsenic trioxide results in decreased methylation of PTPN6 promoter
did not map (561332) decitabine results in decreased methylation of PTPN6 promoter
did not map (762352) Fonofos results in increased methylation of PTPN6 promoter
did not map (827825) PTPN6 protein results in decreased susceptibility to Hydrogen Peroxide
did not map (1120231) Parathion results in increased methylation of PTPN6 promoter
did not map (1372159) terbufos results in increased methylation of PTPN6 promoter
did not map (16080) PTPN11 protein mutant form results in increased susceptibility to 2-(1H-indazol-4-yl)-6-(4-methanesulfonylpiperazin-1-ylmethyl)-4-morpholin-4-ylthieno(3,2-d)pyrimidine
did not map (531985) Curcumin results in decreased expression of PTPN11
did not map (833403) PTPN11 protein mutant form results in increased susceptibility to IC 87114
did not map (1017984) TREM2 results in increased susceptibility to MRK 003
did not map (1018002) TYROBP results in increased susceptibi

CPU times: user 5.2 s, sys: 152 ms, total: 5.35 s
Wall time: 13.5 s


The enrichment function is not perfect at mapping the CTD to BEL. Some things represented don't fit, and others are not the appropriate granularity.

Outstanding interactions to look into:

- (2) 10074-G5 inhibits the reaction [MYC protein binds to MAX protein]
- (87) 10-hydroxywarfarin binds to and results in decreased activity of CYP2C9 protein
- (93) 10-hydroxywarfarin binds to and results in increased activity of NR1I2 protein
- (385) PTGS1 protein results in increased abundance of 11,15-dioxo-9-hydroxy-2,3,4,5-tetranorprostan-1,20-dioic acid
- (495) 1,12-benzoperylene affects the localization of AHR protein

In [13]:
to_jupyter(sialic_acid_graph, height=800)

<IPython.core.display.Javascript object>